In [22]:
%matplotlib widget

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from pathlib import Path

from CWpreprocessing import CSVtoSQLite
from testresults import TestResultsAnalyzer

In [23]:
CSV_FOLDER = Path(r"E:\Python\Project\CWdatafiles")


In [24]:
CSVtoSQLite.clean_counter = 0

if not CSV_FOLDER.exists():
    raise FileNotFoundError(f"Folder not found: {CSV_FOLDER}")

csv_files = list(CSV_FOLDER.glob("*.csv"))

if not csv_files:
    raise FileNotFoundError("No CSV files found")

for csv in csv_files:
    CSVtoSQLite(csv).convert()

print(f"{len(csv_files)} CSV files processed")
print("Database created: CWDatabase.db")


OperationalError: duplicate column name: Q

In [ ]:
DB_PATH = "CWDatabase.db"

def list_tables():
    conn = sqlite3.connect(DB_PATH)
    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table'",
        conn
    )
    conn.close()
    return tables["name"].tolist()


In [ ]:

def load_table(table_name):
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    conn.close()
    return df


In [ ]:
##DEGUG CELL##

DB_PATH = "CWDatabase.db"

conn = sqlite3.connect(DB_PATH)
tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table'",
    conn
)
conn.close()

tables
##DEGUG CELL##

In [ ]:

formatted_tables = tables["name"][
    tables["name"].str.contains("dfFormattedCleanTest")
].tolist()

formatted_tables




In [ ]:
sum_table = next(t for t in formatted_tables if "Sum" in t)

conn = sqlite3.connect(DB_PATH)
df_sum = pd.read_sql(f"SELECT * FROM '{sum_table}'", conn)
conn.close()

df_sum.head()


In [ ]:
df_sum.columns


In [ ]:
plt.figure(figsize=(8,5))
df_sum = df_sum.rename(columns={df_sum.columns[0]: "student_id"})
plt.title("Summative Test Scores (Normalised to 100)")
plt.xlabel("Research ID")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
def get_formatted_tables():
    conn = sqlite3.connect(DB_PATH)
    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table'",
        conn
    )
    conn.close()

    return tables["name"][
        tables["name"].str.contains("dfFormattedCleanTest")
    ].tolist()


def load_table(table_name):
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql(f"SELECT * FROM '{table_name}'", conn)
    conn.close()
    return df


In [ ]:
def plot_table(table_name, title):
    df = load_table(table_name)

    if "student_id" not in df.columns or "score" not in df.columns:
        print("Required columns not found")
        return

    plt.figure(figsize=(9,5))
    plt.bar(df["student_id"], df["score"])
    plt.title(title)
    plt.xlabel("Student ID")
    plt.ylabel("Score (Normalised to 100)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
from matplotlib.widgets import Button

formatted_tables = get_formatted_tables()

sum_table = next(t for t in formatted_tables if "Sum" in t)
formative_tables = [t for t in formatted_tables if "Sum" not in t]

fig, ax = plt.subplots(figsize=(6,4))
ax.axis("off")

ax_sum = plt.axes([0.3, 0.6, 0.4, 0.15])
ax_form = plt.axes([0.3, 0.35, 0.4, 0.15])

btn_sum = Button(ax_sum, "Show Summative Results")
btn_form = Button(ax_form, "Show Formative Results")


In [ ]:
def show_summative(event):
    plot_table(sum_table, "Summative Test Results")


def show_formative(event):
    for table in formative_tables:
        plot_table(table, f"Formative Results: {table.split('_')[0]}")


btn_sum.on_clicked(show_summative)
btn_form.on_clicked(show_formative)

plt.show()
